In [63]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

In [64]:
# Создаём сэмпл
n_samples = 500

age_owner = np.random.choice(90, n_samples) + 21
length = np.random.choice(120, n_samples) + 15
width = np.random.choice(80, n_samples) + 10

price = length * width * 100

data = pd.DataFrame({'age_owner': age_owner, 'length': length, 'width': width, 'price': price})
data.head(5)

,age_owner,length,width,price
0,81,78,10,78000
1,27,63,33,207900
2,26,65,65,422500
3,89,54,21,113400
4,102,105,35,367500


In [65]:
from sklearn.metrics import mean_absolute_error

X = data[['age_owner', 'length', 'width']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['age_owner', 'length', 'width']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-257.48542082 4735.53667373 7413.81058854]
Bias: -332932.76302401826
Error: 55541.44685343691


In [66]:
X = data[['length', 'width']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['length', 'width']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [4740.67725968 7414.33397371]
Bias: -350553.9474441905
Error: 55844.50661606447


In [67]:
# Создаем новый признак
data['mult'] = data['length'] * data['width']
data.head(5)

,age_owner,length,width,price,mult
0,81,78,10,78000,780
1,27,63,33,207900,2079
2,26,65,65,422500,4225
3,89,54,21,113400,1134
4,102,105,35,367500,3675


In [68]:
X = data[['mult']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['mult']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [100.]
Bias: -1.1641532182693481e-10
Error: 5.916081136092544e-11


## ** Домашнее задание **

По образу практики, попробуйте создать искусственный датасет с лишними столбцами.

Для начала создам датафрейм, содержащий инофрмацию о людях:

* Возраст - 2 - 90 age
* Пол - m(male) / f(female)
* Вес - 7 - 210 kg
* Рост - 60 - 225 sm
* Фототип кожи - Scandinavian, light-skinned European, central european, Mediterranean, Asian, African.
* Размер обуви - 34 - 52 sm
* Умер - y(yes)/n(no)

По данному датафрейму, буду пытаться понять влияет ли масса тела, да и просто его размеры на смерть.

In [93]:
YES = 'y'
NO = 'm'
MALE = 'm'
FEMALE = 'f'
n_samples = 5000

skin_phototypes = ['scandinavian', 'light_skinned_European', 'central_european', 'mediterranean', 'asian', 'african']
skin_phototype = [skin_phototypes[i] for i in np.random.choice(6, n_samples)]

age = np.random.choice(88, n_samples) + 3
weight = np.random.choice(141, n_samples) + 10
height = np.random.choice(171, n_samples) + 50
sex = [FEMALE if i > 1 else MALE for i in np.random.choice(3, n_samples)]
died = [YES if i < 1 else NO for i in np.random.choice(50, n_samples)]
shoe_size = np.random.choice(35, n_samples) + 18

In [94]:
target_label = 'died'
labels = ['age', 'weight', 'height', 'sex', 'shoe_size', 'skin_phototype']
all_labels = [*labels, target_label]
data = [age, weight, height, sex, shoe_size, skin_phototype, died]

df = pd.DataFrame(
    data=dict(zip(all_labels, data)),
)

df.head()

,age,weight,height,sex,shoe_size,skin_phototype,died
0,56,71,99,m,26,light_skinned_European,m
1,53,147,102,m,36,scandinavian,m
2,8,73,88,m,25,african,m
3,24,14,97,f,40,asian,m
4,45,138,79,m,39,light_skinned_European,m


С горем пополам вроде поулчился нужный датасет.

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
age               5000 non-null int32
weight            5000 non-null int32
height            5000 non-null int32
sex               5000 non-null object
shoe_size         5000 non-null int32
skin_phototype    5000 non-null object
died              5000 non-null object
dtypes: int32(4), object(3)
memory usage: 195.4+ KB


Что бы было посложнее, решил не делать все значения числовыми, а перевести самостоятельно.

Каждый значение 'n' мы заменим на 0, 'y' на 1, 'f' на 0, 'm' на 1.

Каждый значение из skin_phototype мы заменим на число от 1 до длины списка значений skin_phototypes

In [96]:
def change_y_n(val):
    if val == NO:
        return 0
    elif val == YES:
        return 1
    return val

df['died'] = df['died'].apply(change_y_n)

df.head()

,age,weight,height,sex,shoe_size,skin_phototype,died
0,56,71,99,m,26,light_skinned_European,0
1,53,147,102,m,36,scandinavian,0
2,8,73,88,m,25,african,0
3,24,14,97,f,40,asian,0
4,45,138,79,m,39,light_skinned_European,0


In [97]:
def change_skin_phototype(val):
    try:
        return skin_phototypes.index(val) + 1 # count from 1
    except:
        return -1
    return val

df['skin_phototype'] = df['skin_phototype'].apply(change_skin_phototype)

df.head()

,age,weight,height,sex,shoe_size,skin_phototype,died
0,56,71,99,m,26,2,0
1,53,147,102,m,36,1,0
2,8,73,88,m,25,6,0
3,24,14,97,f,40,5,0
4,45,138,79,m,39,2,0


In [98]:
def change_sex(val):
    if val == FEMALE:
        return 0
    elif val == MALE:
        return 1
    return val

df['sex'] = df['sex'].apply(change_sex)

df.head()

,age,weight,height,sex,shoe_size,skin_phototype,died
0,56,71,99,1,26,2,0
1,53,147,102,1,36,1,0
2,8,73,88,1,25,6,0
3,24,14,97,0,40,5,0
4,45,138,79,1,39,2,0


Все колонки с значением "object", переведены к числовому типу.

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
age               5000 non-null int32
weight            5000 non-null int32
height            5000 non-null int32
sex               5000 non-null int64
shoe_size         5000 non-null int32
skin_phototype    5000 non-null int64
died              5000 non-null int64
dtypes: int32(4), int64(3)
memory usage: 195.4 KB


Так как вроде я больше не мудрил и реузльтат схож с примером можно приступать к импровизации.

In [120]:
x = df[[*labels, 'sex', 'died']]
y = df[target_label]

In [121]:
model = LinearRegression().fit(x, y)
print(f'Weights: {model.coef_}')
print(f'Bias: {model.intercept_}')

pred_values = model.predict(x)
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [-2.42141508e-19  7.58941521e-19 -4.33680869e-19  1.98811973e-16
 -1.26546722e-18 -1.26716129e-18 -1.69816765e-16  1.00000000e+00]
Bias: 2.7755575615628914e-17
Error: 4.422074269874227e-17


тут пришлось разбираться почему все нахрен перестало работать, так как он не хотел воспринимать значение 'yes' пришлось менять на y.

Попробую удалить лишнее значение, такое как shoe_size.

In [122]:
x = df[['age', 'weight', 'height', 'sex', 'skin_phototype']]
y = df[target_label]

In [123]:
model = LinearRegression().fit(x, y)
print(f'Weights: {model.coef_}')
print(f'Bias: {model.intercept_}')

pred_values = model.predict(x)
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [-2.76350184e-05  8.98705970e-05  4.74399405e-05  3.33760852e-03
  4.45197093e-04]
Bias: 0.00534461221776323
Error: 0.04183848880615871


Почему то как то странно поменялись данные, но порпоруем убрать еще 'age', 'sex', 'skin_phototype'.

In [124]:
x = df[['weight', 'height',]]
y = df[target_label]

In [125]:
model = LinearRegression().fit(x, y)
print(f'Weights: {model.coef_}')
print(f'Bias: {model.intercept_}')

pred_values = model.predict(x)
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [9.08685338e-05 4.69898664e-05]
Bias: 0.007850262729244783
Error: 0.041845509930155754


Теперь как показанно в примере создаем новый индекс, в моем случаее это масса тела.

In [126]:
df['body weight'] = df['weight'] / pow(df['height'] / 100, 2)

In [127]:
x = df[['body weight']]
y = df[target_label]

In [128]:
model = LinearRegression().fit(x, y)
print(f'Weights: {model.coef_}')
print(f'Bias: {model.intercept_}')

pred_values = model.predict(x)
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [3.7454919e-06]
Bias: 0.02112184022155591
Error: 0.041883863581774275


Не совсем догнал что нам доют, данные по средним ошибкам, но после обработки мы можем провести в разы больше проверок, чем раньше.

Скорее всего по нормальным датасетам, с настоящими данными. полученные материлы будут более понятны и вменяемы.